In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd

df = pd.read_csv('gdrive/My Drive/train.csv')

In [0]:
df = df.dropna()

In [0]:
from keras.preprocessing.text import Tokenizer

In [0]:
tokenizer = Tokenizer()

In [0]:
X1 = df['question1']
X2 = df['question2']
Y = df['is_duplicate']

In [0]:
tokenizer.fit_on_texts(X1 + ' ' + X2)

In [0]:
X1 = tokenizer.texts_to_sequences(X1)
X2 = tokenizer.texts_to_sequences(X2)

In [0]:
import numpy as np

Y = np.array(Y)

In [0]:
max_len1 = 0
for x in X1:
  if(len(x)>max_len1):
    max_len1 = len(x)

max_len2 = 0
for x in X2:
  if(len(x)>max_len2):
    max_len2 = len(x)

In [96]:
print(max_len1, max_len2)

127 237


In [0]:
from keras.preprocessing.sequence import pad_sequences

X1 = pad_sequences(X1, maxlen = max_len1, padding='post')
X2 = pad_sequences(X2, maxlen = max_len2, padding='post')

In [0]:
vocab_size = len(tokenizer.word_index) + 1

In [85]:
from keras.layers import Input, Embedding, Dropout, LSTM, Dense
from keras.models import Model
from keras.layers.merge import add

inputs1 = Input(shape=(max_len1,))
emb1 = Embedding(vocab_size, 256, mask_zero=True)(inputs1)
se2 = Dropout(0.5)(emb1)
se3 = LSTM(256)(se2)

inputs2 = Input(shape=(max_len2,))
emb2 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
fe2 = Dropout(0.5)(emb2)
fe3 = LSTM(256)(fe2)

decoder1 = add([fe3, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(1, activation='sigmoid')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 237)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 127)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 237, 256)     24472576    input_3[0][0]                    
_____________________________________________________________________________

In [0]:
n = int(len(X1)*0.7)

X1_train = X1[:n]
X1_test = X1[n:]

X2_train = X2[:n]
X2_test = X2[n:]

Y_train = Y[:n]
Y_test = Y[n:]

In [106]:
model.fit([X1_train, X2_train], Y_train, epochs = 10, validation_split=0.1)




Train on 254700 samples, validate on 28300 samples
Epoch 1/10





254700/254700 [==============================] - 6700s 26ms/step - loss: 0.5106 - acc: 0.7494 - val_loss: 0.4751 - val_acc: 0.7722
Epoch 2/10
153696/254700 [=================>............] - ETA: 41:58 - loss: 0.4207 - acc: 0.8050

KeyboardInterrupt: ignored

In [108]:
history = model.evaluate([X1_test, X2_test], Y_test)

121287/121287 [==============================] - 976s 8ms/step


[0.46690171414816045, 0.780627767197797]